# 1. Contexto de Negócio:

"House Flipping" é uma modalidade de investimento imobiliário, que compreende (i) a compra de imóveis descontados, (ii) reforma de custo reduzido com foco em valor de mercado, e (iii) posterior venda em curto espaço de tempo. Tradicionalmente esta operação é focada em imóveis residenciais.

A grande ideia por trás desse investimento é adquirir imóveis bastante descontados quando o vendedor tem urgência no negócio (casos de dívidas, divórcio, falecimento, necessidade de liberar capital, etc), priorizando excelente localização. Para o contexto do "House Flipping", boa localização seria aquela que permitiria o giro rápido do imóvel, ou seja, bairros de alta liquidez. 

Seguindo o princípio da oferta e da procura, bairros de alta liquidez nos centros urbanos são, em geral, aqueles com maior preço por metro quadrado. Por este motivo, uma análise dos preços de venda de imóveis é um excelente ponto de partida para um investidor desse tipo selecionar sua área de atuação.  

# 2. Objetivo da análise:

O objetivo desta análise é fazer um panorama geral de preços de venda de imóveis residenciais na cidade de São Paulo. 

Para este intuito, serão analisados os dados de todas as transações imobiliárias para as quais houve recolhimento de ITBI (Imposto sobre a Transmissão de Bens Imóveis), no período de 2022 a 2025 nessa cidade.

As principais perguntas que se propõe responder são:
- Quais são os top 10 bairros de maior preço por metro quadrado na cidade de São Paulo?
- Como tem sido a variação desses valores nos últimos 3 anos? Que bairros apresentam maior valorização?
- Que características dos imóveis (tipologia, número de quartos, vaga de garagem, com/sem elevador, etc) estão relacionadas a maior valorização deles?
- Existe sazonalidade no mercado imobiliário desta cidade? Quais são os meses mais indicados para compra descontada?
- Como é a aceitação dos vendedores em relação a financiamento? Em média, qual percentual do valor de venda é financiado? 

Observação: Inicialmente o objetivo era estudar a cidade de Juiz de Fora - MG, onde resido. No entanto, tal objetivo não foi possível pela falta de disponibilidade de informações granulares no portal de dados de ITBI dessa cidade.

# 3. Busca e coleta de dados

## 3.1 Fontes de Dados

### 3.1.1 Dados ITBI

A fonte dos dados de ITBI utilizados nesta análise é a Prefeitura de São Paulo, através do endereço: <https://prefeitura.sp.gov.br/web/fazenda/w/acesso_a_informacao/31501>

**URL das bases de dados em formato xlsx:**

2025: https://prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/itbi/GUIAS%20DE%20ITBI%20PAGAS%20%2825112025%29.xlsx

2024: https://prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/itbi/GUIAS-DE-ITBI-PAGAS-2024.xlsx

2023: https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/XLSX/GUIAS-DE-ITBI-PAGAS-2023.xlsx

2022: https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/XLSX/GUIAS_DE_ITBI_PAGAS_12-2022.xlsx 

Cada uma das URLs acima descritas apresenta os dados de um ano calendário. Dentro do arquivo xlsx os dados de cada um dos meses são reportados em abas diferentes. Para o ano de 2025 estão disponíveis dados até o mês de outubro.

As últimas abas dos arquivos contém informações complementares que serão utilizadas no glossário de dados e nas tabelas dimensão que serão descritas no capítulo de modelagem. 


### 3.1.2 Dados de CEP

As informações a respeito dos CEPs foram obtidas no site CEP Aberto, disponível em: https://www.cepaberto.com/downloads/new

Cidades e Municípios: https://www.cepaberto.com/downloads.csv?name=cities

Estados: https://www.cepaberto.com/downloads.csv?name=states


CEPs do estado de São Paulo:

https://www.cepaberto.com/downloads.csv?name=SP&part=1

https://www.cepaberto.com/downloads.csv?name=SP&part=2

https://www.cepaberto.com/downloads.csv?name=SP&part=3

https://www.cepaberto.com/downloads.csv?name=SP&part=4

https://www.cepaberto.com/downloads.csv?name=SP&part=5



O armazenamento dessas bases em nuvem está descrito nos scripts abaixo.

## 3.2 Ingestão de dados e criação de tabelas

### 3.2.1 Tabela consolidada com dados de ITBI (2022 a 2025):

In [0]:
#Instalar biblioteca openpyxl para ler arquivo em formato xlsx
 
%pip install openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
#Reinicia o ambiente Python para garantir que a biblioteca esteja disponível

%restart_python

In [0]:
# Cria tabela com dados de ITBI 2025

import requests
import pandas as pd
import re

url = (
    'https://prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/itbi/'
    'GUIAS%20DE%20ITBI%20PAGAS%20%2825112025%29.xlsx'
)

# Baixa o arquivo da url e salva em um arquivo temporário no cluster
response = requests.get(url)
with open('/tmp/seuarquivo.xlsx', 'wb') as f:
    f.write(response.content)

# Lê as 10 primeiras abas do arquivo Excel (jan a out) para gerar um dicionário Python (chave - valor), contendo nome da aba e conteúdo gravado em um dataframe
sheets = pd.read_excel(
    '/tmp/seuarquivo.xlsx',
    sheet_name=list(range(10))
)

# Seleciona apenas as 28 primeiras colunas de cada aba (dados para além dessas são lixo) e guarda em uma lista de dataframes
dfs = [df.iloc[:, :28].copy() for df in sheets.values()]

# Copia o header do primeiro df em todos os demais para conseguir concatená-los (dados originais tem headers despadronizados)
columns = dfs[0].columns.tolist()
for df in dfs:
    df.columns = columns
pdf_all = pd.concat(dfs, ignore_index=True)

# Converte todas as colunas que contêm texto para tipo de dado string
for col in pdf_all.select_dtypes(include=['object']).columns:
    pdf_all[col] = pdf_all[col].astype(str)

#Define função que limpa os nomes das colunas: tira espaços e caracteres especiais
def clean_column(name):
    return re.sub(r'[ ,;{}()\n\t=/\.]', '_', str(name)).replace('°', 'o').replace('%', 'pct')

#Chama a função clean_column definida anterioremente para limpar os nomes das colunas de pdf_all, em preparação para convertê-lo para DataFrame do Spark
pdf_all.columns = [clean_column(col) for col in pdf_all.columns]

# Converte o DataFrame do Pandas para um DataFrame do Spark
df = spark.createDataFrame(pdf_all)

# Deleta a tabela, caso já exista
table_name = "mvp_engdados_puc.bronze.guias_itbi_2025"
spark.sql(f"DROP TABLE IF EXISTS {table_name} PURGE")

# Conta número de linhas do DataFrame e mostra 100 linhas dele
num_linhas = df.count()
print(f"Número de linhas no Spark: {num_linhas}")
display(df.limit(20))

# Salva o DataFrame do Spark em uma tabela persistida no DBFS
df.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(table_name)

# Os headers de cada aba do Excel original estão misturados no corpo dos dados. Esta limpeza será feita na etapa de qualidade.

Número de linhas no Spark: 188073


No_do_Cadastro__SQL_,Nome_do_Logradouro,Número,Complemento,Bairro,Referência,CEP,Natureza_de_Transação,Valor_de_Transação__declarado_pelo_contribuinte_,Data_de_Transação,Valor_Venal_de_Referência,Proporção_Transmitida__pct_,Valor_Venal_de_Referência__proporcional_,Base_de_Cálculo_adotada,Tipo_de_Financiamento,Valor_Financiado,Cartório_de_Registro,Matrícula_do_Imóvel,Situação_do_SQL,Área_do_Terreno__m2_,Testada__m_,Fração_Ideal,Área_Construída__m2_,Uso__IPTU_,Descrição_do_uso__IPTU_,Padrão__IPTU_,Descrição_do_padrão__IPTU_,ACC__IPTU_
30106900789,AV CARLOS QUEIROZ TELLES,99999,Apto 163 Bloc L,IBIRAPUERA,nan,5704150.0,1.Compra e venda,625100.0,2024-12-20 00:00:00,1.2318299E7,0.01,1231.8299,625100.0,1.Sistema Financeiro de Habitação,379599.36,15º Cartório de Registro de Imóvel,270983,Ativo Territorial,8066,8.0,1.0,0,0,TERRENO,0,TERRENO,nan
17130212951,R DR LUIZ MIGLIANO,1870,APTO 223 TORRE E,COND MONUMENTO SP,nan,5711001.0,1.Compra e venda,573915.07,2024-12-20 00:00:00,2.0652147E7,0.07,14456.5029,573915.07,1.Sistema Financeiro de Habitação,450000.0,18º Cartório de Registro de Imóvel,213973,Ativo Territorial,59923,256.24,0.5155,0,0,TERRENO,0,TERRENO,nan
9900500245,R CAP JOSE INACIO DO ROSARIO,133,AP 2015 TORRE B,LAPA,nan,5038070.0,1.Compra e venda,288435.0,2024-12-20 00:00:00,8.5718062E7,0.01,8571.8062,288435.0,2.Minha Casa Minha Vida,230746.95,10º Cartório de Registro de Imóvel,173502,Ativo Predial,22903,0.0,1.0,11382,71,ESCOLA,94,"TEMPLO, CLUBE, GINÁSIO OU ESTÁDIOESPORTIVO, MUSEU, HIPÓDROMO, CINEMA,TEATRO, AEROPORTO, ESTAÇÕES, ETC.",2002.0
2601502142,R MARCOS ARRUDA,729,AP 2411 T A,BELENZINHO,nan,3020000.0,1.Compra e venda,237000.0,2024-12-20 00:00:00,4.352435E7,0.01,4352.435,237000.0,1.Sistema Financeiro de Habitação,188419.9,7º Cartório de Registro de Imóvel,240242,Ativo Predial,13221,137.95,1.0,33835,71,ESCOLA,44,COMERCIAL VERTICAL,1959.0
1904806211,R NEWTON PRADO,767,AP 136B E VG,COND ESTILO,TORRE B,1127000.0,4.Arrematação (em leilão ou hasta pública),251576.26,2024-12-18 00:00:00,0.0,100.0,0.0,251576.26,nan,0.0,8º Cartório de Registro de Imóvel,180348,Ativo Predial,5199,55.5,0.0048,95,20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),22,RESIDENCIAL VERTICAL,2014.0
8241900546,R MIRAGAIA,174,AP STUDIO 205,nan,LA VIDA BUTANTA,5511020.0,1.Compra e venda,400000.0,2024-12-23 00:00:00,209275.0,0.7,1464.925,400000.0,1.Sistema Financeiro de Habitação,284400.0,18º Cartório de Registro de Imóvel,284832,Ativo Predial,997,26.0,0.007,60,85,FLAT DE USO COMERCIAL (SEMELHANTE À HOTEL),22,RESIDENCIAL VERTICAL,2023.0
17207900774,R MANUEL NAJERA,246,LT B,CID JULIA,nan,4424020.0,1.Compra e venda,650000.0,2024-12-23 00:00:00,441031.0,100.0,441031.0,650000.0,1.Sistema Financeiro de Habitação,370000.0,11º Cartório de Registro de Imóvel,381072,Ativo Predial,138,6.0,1.0,187,10,RESIDÊNCIA,12,RESIDENCIAL HORIZONTAL,1998.0
16200137721,R MIGUEL YUNES,99999,APTO 202 TORRE 2,nan,nan,4444000.0,1.Compra e venda,249513.88,2024-12-20 00:00:00,4.2068483E7,0.1,42068.483,249513.88,1.Sistema Financeiro de Habitação,197636.63,11º Cartório de Registro de Imóvel,506492,Ativo Predial,42369,232.53,1.0,320,84,OUTRAS EDIFICAÇÕES DO TIPO COM UTILIZAÇÃO MÚLTIPLA,93,"TEMPLO, CLUBE, GINÁSIO OU ESTÁDIOESPORTIVO, MUSEU, HIPÓDROMO, CINEMA,TEATRO, AEROPORTO, ESTAÇÕES, ETC.",1988.0
19704200071,AV MARQ DE SAO VICENTE,995,AG BRA ALA B 1106,nan,nan,1139003.0,1.Compra e venda,249397.21,2024-12-24 00:00:00,2.8223732E7,0.22,62092.2104,249397.21,1.Sistema Financeiro de Habitação,194380.15,15º Cartório de Registro de Imóvel,291184,Ativo Predial,9331,37.6,1.0,5375,64,"OUTRAS EDIFICAÇÕES DO TIPO, COM UTILIZAÇÃO MÚLTIPLA",83,"OFICINA, POSTO DE SERVIÇO, ARMAZÉM/DEPÓSITO, INDÚSTRIA",1988.0
2601502142,R MARCOS ARRUDA,729,AP 1505 T C,BELENZINHO,nan,3020000.0,1.Compra e venda,234000.0,2024-12-20 00:00:00,4.352435E7,0.01,4352.435,234000.0,2.Minha Casa Minha Vida,186635.86,7º Cartório de Registro de Imóvel,240242,Ativo Predial,13221,137.95,1.0,33835,71,ESCOLA,44,COMERCIAL VERTICAL,1959.0


In [0]:

# Processamento idem anterior para criar as tabelas de itbi pagos de 2022 a 20224

import requests
import pandas as pd
import re

urls = [
    'https://prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/itbi/GUIAS-DE-ITBI-PAGAS-2024.xlsx',
    'https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/XLSX/GUIAS-DE-ITBI-PAGAS-2023.xlsx',
    'https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/XLSX/GUIAS_DE_ITBI_PAGAS_12-2022.xlsx',
]

#Define função que limpa os nomes das colunas: tira espaços e caracteres especiais
def clean_column(name):
    return re.sub(r'[ ,;{}()\n\t=/\.]', '_', str(name)).replace('°', 'o').replace('%', 'pct')

# Para os anos de 2022 a 2024:
for idx, url in enumerate(urls, start=1):
    
    # Baixa o arquivo da url e salva em um arquivo temporário no cluster
    response = requests.get(url)
    file_path = f'/tmp/seuarquivo_{idx}.xlsx'
    with open(file_path, 'wb') as f:
        f.write(response.content)

    # Lê as 12 primeiras abas do arquivo Excel (jan a dez)
    sheets = pd.read_excel(
        file_path,
        sheet_name=list(range(12))
    )

    # Seleciona apenas as 28 primeiras colunas de cada aba (dados para além dessas são lixo)
    dfs = [df.iloc[:, :28].copy() for df in sheets.values()]

    # Copia um header selecionado padrão em todos os demais para conseguir concatená-los (dados originais tem headers despadronizados)
    columns = dfs[1].columns.tolist()
    for df in dfs:
        df.columns = columns
    pdf_all = pd.concat(dfs, ignore_index=True)

    # Converte todas as colunas que contêm texto para tipo de dado string
    for col in pdf_all.select_dtypes(include=['object']).columns:
        pdf_all[col] = pdf_all[col].astype(str)

    #Chama a função clean_column definida anteriormente para limpar os nomes das colunas de pdf_all, em preparação para convertê-lo para DataFrame do Spark
    pdf_all.columns = [clean_column(col) for col in pdf_all.columns]

    # Converte o DataFrame do Pandas para um DataFrame do Spark
    df = spark.createDataFrame(pdf_all)

    # Deleta a tabela, caso já exista
    table_name = f"mvp_engdados_puc.bronze.guias_itbi_{idx}"
    spark.sql(f"DROP TABLE IF EXISTS {table_name} PURGE")

    # Conta número de linhas do DataFrame e mostra 100 linhas dele
    num_linhas = df.count()
    print(f"Número de linhas no Spark: {num_linhas}")
    display(df.limit(20))

    # Salva o DataFrame do Spark em uma tabela persistida no DBFS
    df.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(table_name)

# Os headers de cada aba do Excel original estão misturados no corpo dos dados. Esta limpeza será feita na etapa de qualidade.


Número de linhas no Spark: 217744


No_do_Cadastro__SQL_,Nome_do_Logradouro,Número,Complemento,Bairro,Referência,CEP,Natureza_de_Transação,Valor_de_Transação__declarado_pelo_contribuinte_,Data_de_Transação,Valor_Venal_de_Referência,Proporção_Transmitida__pct_,Valor_Venal_de_Referência__proporcional_,Base_de_Cálculo_adotada,Tipo_de_Financiamento,Valor_Financiado,Cartório_de_Registro,Matrícula_do_Imóvel,Situação_do_SQL,Área_do_Terreno__m2_,Testada__m_,Fração_Ideal,Área_Construída__m2_,Uso__IPTU_,Descrição_do_uso__IPTU_,Padrão__IPTU_,ACC__IPTU_,ACC__IPTU__1
101101181,R VINTE E CINCO DE JANEIRO,151,AP 44,CENTRO,ED JOSE DE ANCHIETA,1103000,1.Compra e venda,190000,2024-01-23 00:00:00,172186,100,172186,190000,nan,0,5º Cartório de Registro de Imóvel,85213,Ativo Predial,3050,100,0.0027,42,20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),21,RESIDENCIAL VERTICAL,2008
101103395,R VINTE E CINCO DE JANEIRO,151,AP 26,CENTRO,ED ANTONIO VIEIRA,1103000,1.Compra e venda,260000,2024-01-17 00:00:00,220411,100,220411,260000,nan,0,5º Cartório de Registro de Imóvel,85103,Ativo Predial,3050,100,0.0034,54,20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),21,RESIDENCIAL VERTICAL,2008
101700091,R POSSIDONIO INACIO,151,E 159,nan,nan,1103030,1.Compra e venda,650000,2024-01-12 00:00:00,374063,100,374063,650000,nan,0,5º Cartório de Registro de Imóvel,59031,Ativo Predial,120,14.2,1,197,42,LOJA E RESIDÊNCIA (PREDOMINÂNCIA COMERCIAL),42,COMERCIAL VERTICAL,1949
101904959,AV CASPER LIBERO,501,AP 702,STA EFIGENIA,MOVVA SP,1033001,1.Compra e venda,170000,2024-01-12 00:00:00,211008,100,211008,211008,nan,0,5º Cartório de Registro de Imóvel,106709,Ativo Predial,1150,19.83,0.0038,35,20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),22,RESIDENCIAL VERTICAL,2020
102007276,R WASHINGTON LUIS,196,AP 2006,SETIN DOWNTOWN,ESTACAO DA LUZ,1033010,1.Compra e venda,230000,2024-02-01 00:00:00,173708,100,173708,230000,nan,0,5º Cartório de Registro de Imóvel,103697,Ativo Predial,935,25.59,0.0036,31,20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),22,RESIDENCIAL VERTICAL,2018
102307040,R FLORENCIO DE ABREU,752,AP 615,COND HOTEL FLORENCIO,DE ABREU,1030001,1.Compra e venda,248400,2024-01-15 00:00:00,214179,100,214179,248400,nan,0,5º Cartório de Registro de Imóvel,105904,Ativo Predial,1800,20,0.0032,42,80,"HOTEL, PENSÃO OU HOSPEDARIA",43,COMERCIAL VERTICAL,2019
102307962,R FLORENCIO DE ABREU,752,AP 1011,COND HOTEL FLORENCIO,DE ABREU,1030001,1.Compra e venda,237600,2024-01-31 00:00:00,214179,100,214179,237600,nan,0,5º Cartório de Registro de Imóvel,105996,Ativo Predial,1800,20,0.0032,42,80,"HOTEL, PENSÃO OU HOSPEDARIA",43,COMERCIAL VERTICAL,2019
102307989,R FLORENCIO DE ABREU,752,AP 1013,COND HOTEL FLORENCIO,DE ABREU,1030001,1.Compra e venda,245700,2024-01-31 00:00:00,214179,100,214179,245700,nan,0,5º Cartório de Registro de Imóvel,105998,Ativo Predial,1800,20,0.0032,42,80,"HOTEL, PENSÃO OU HOSPEDARIA",43,COMERCIAL VERTICAL,2019
102307997,R FLORENCIO DE ABREU,752,AP 1014,COND HOTEL FLORENCIO,DE ABREU,1030001,1.Compra e venda,245700,2024-01-31 00:00:00,214179,100,214179,245700,nan,0,5º Cartório de Registro de Imóvel,105999,Ativo Predial,1800,20,0.0032,42,80,"HOTEL, PENSÃO OU HOSPEDARIA",43,COMERCIAL VERTICAL,2019
102308004,R FLORENCIO DE ABREU,752,AP 1015,COND HOTEL FLORENCIO,DE ABREU,1030001,1.Compra e venda,237600,2024-01-31 00:00:00,214179,100,214179,237600,nan,0,5º Cartório de Registro de Imóvel,106000,Ativo Predial,1800,20,0.0032,42,80,"HOTEL, PENSÃO OU HOSPEDARIA",43,COMERCIAL VERTICAL,2019


Número de linhas no Spark: 175459


No_do_Cadastro__SQL_,Nome_do_Logradouro,Número,Complemento,Bairro,Referência,CEP,Natureza_de_Transação,Valor_de_Transação__declarado_pelo_contribuinte_,Data_de_Transação,Valor_Venal_de_Referência,Proporção_Transmitida__pct_,Valor_Venal_de_Referência__proporcional_,Base_de_Cálculo_adotada,Tipo_de_Financiamento,Valor_Financiado,Cartório_de_Registro,Matrícula_do_Imóvel,Situação_do_SQL,Área_do_Terreno__m2_,Testada__m_,Fração_Ideal,Área_Construída__m2_,Uso__IPTU_,Descrição_do_uso__IPTU_,Padrão__IPTU_,Descrição_do_padrão__IPTU_,ACC__IPTU_
13944000322,AV NORDESTINA,3010,AP 918 TORRE 2,VL CURUCA,nan,8032000.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,1.582887E7,0.2,31657.74,264000.0,1.Sistema Financeiro de Habitação,166320.0,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
6726500084,R JURIPIRANGA,70,ANT 16,PARQUE VITORIA,nan,2269090.0,1.Compra e venda,800000.0,2022-12-23 00:00:00,846885.0,100.0,846885.0,846885.0,1.Sistema Financeiro de Habitação,400000.0,15º Cartório de Registro de Imóvel,87025,Ativo Predial,210,10.0,1.0,277,10,RESIDÊNCIA,12,RESIDENCIAL HORIZONTAL,2009.0
13944000322,AV NORDESTINA,3010,AP 504 SUB 2 BL 2,VL CURUCA,nan,8032000.0,1.Compra e venda,203700.0,2022-12-22 00:00:00,1.582887E7,0.01,1582.887,203700.0,1.Sistema Financeiro de Habitação,144059.51,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
5614900926,R ANTONIO DE BARROS,99999,AP 614 BL 3,TATUAPE,nan,3401001.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,3403192.0,0.08,2722.5536,264000.0,1.Sistema Financeiro de Habitação,183400.0,9º Cartório de Registro de Imóvel,336376,Ativo Territorial,1000,20.0,1.0,0,0,TERRENO,0,TERRENO,nan
13944000322,AV NORDESTINA,3010,AP 305 SUB 1 BL 1,VL CURUCA,nan,8032000.0,1.Compra e venda,200550.0,2022-12-22 00:00:00,1.582887E7,0.01,1582.887,200550.0,1.Sistema Financeiro de Habitação,100433.0,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0.0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 2102 T A,PIRITUBA,nan,2938000.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,2.3872922E7,0.02,4774.5844,264000.0,1.Sistema Financeiro de Habitação,208635.41,16º Cartório de Registro de Imóvel,185609,Ativo Territorial,27500,172.0,1.0,0,0,TERRENO,0,TERRENO,nan
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 1701 T B,PIRITUBA,nan,2938000.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,2.3872922E7,0.02,4774.5844,264000.0,1.Sistema Financeiro de Habitação,191064.56,16º Cartório de Registro de Imóvel,185610,Ativo Territorial,27500,172.0,1.0,0,0,TERRENO,0,TERRENO,nan
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 1712 T A,PIRITUBA,nan,2938000.0,1.Compra e venda,260000.0,2022-12-22 00:00:00,2.3872922E7,0.02,4774.5844,260000.0,1.Sistema Financeiro de Habitação,165514.76,16º Cartório de Registro de Imóvel,185608,Ativo Territorial,27500,172.0,1.0,0,0,TERRENO,0,TERRENO,nan
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 505 T A,PIRITUBA,nan,2938000.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,2.3872922E7,0.02,4774.5844,264000.0,1.Sistema Financeiro de Habitação,211200.0,16º Cartório de Registro de Imóvel,185610,Ativo Territorial,27500,172.0,1.0,0,0,TERRENO,0,TERRENO,nan
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 2104 T A,PIRITUBA,nan,2938000.0,1.Compra e venda,264000.0,2022-12-22 00:00:00,2.3872922E7,0.02,4774.5844,264000.0,1.Sistema Financeiro de Habitação,211200.0,16º Cartório de Registro de Imóvel,185608,Ativo Territorial,27500,172.0,1.0,0,0,TERRENO,0,TERRENO,nan


Número de linhas no Spark: 163903


No_do_Cadastro__SQL_,Nome_do_Logradouro,Número,Complemento,Bairro,Referência,CEP,Natureza_de_Transação,Valor_de_Transação__declarado_pelo_contribuinte_,Data_de_Transação,Valor_Venal_de_Referência,Proporção_Transmitida__pct_,Valor_Venal_de_Referência__proporcional_,Base_de_Cálculo_adotada,Tipo_de_Financiamento,Valor_Financiado,Cartório_de_Registro,Matrícula_do_Imóvel,Situação_do_SQL,Área_do_Terreno__m2_,Testada__m_,Fração_Ideal,Área_Construída__m2_,Uso__IPTU_,Descrição_do_uso__IPTU_,Padrão__IPTU_,ACC__IPTU_,ACC__IPTU__1
17121501499,R FRANCISCO PESSOA,695,AP 94 E 2 VGS,VL ANDRADE,ED POESIA,5727230,1.Compra e venda,720000.0,2021-12-21T00:00:00.000Z,538738.0,100.0,538738.0,720000.0,1.Sistema Financeiro de Habitação,250000.0,11º Cartório de Registro de Imóvel,364522,Ativo Predial,8263,109.3,0.0056,178,20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),22,RESIDENCIAL VERTICAL,2009.0
6410900473,R CURUCA,684,TORRE 01 AP 1111,VILA MARIA,nan,2120001,1.Compra e venda,253000.0,2021-12-23T00:00:00.000Z,7065368.0,0.02,1413.0736,253000.0,1.Sistema Financeiro de Habitação,202400.0,17º Cartório de Registro de Imóvel,202400,Ativo Predial,3800,80.0,1.0,4483,50,INDÚSTRIA,83,"OFICINA, POSTO DE SERVIÇO, ARMAZÉM/DEPÓSITO, INDÚSTRIA",1978.0
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,TORRE A AP 702,PIRITUBA,nan,2938000,1.Compra e venda,264000.0,2021-12-23T00:00:00.000Z,1.8918426E7,0.02,3783.6852,264000.0,1.Sistema Financeiro de Habitação,194464.25,16º Cartório de Registro de Imóvel,185608,Ativo Territorial,27500,172.0,1.0,0,0,TERRENO,0,TERRENO,null
2806802113,AV PRES WILSON,30,AP 1502 T 3,nan,nan,3107000,1.Compra e venda,264000.0,2021-12-23T00:00:00.000Z,1.3087429E7,0.14,18322.4006,264000.0,1.Sistema Financeiro de Habitação,170993.91,7º Cartório de Registro de Imóvel,212252,Ativo Territorial,5591,140.73,1.0,0,0,TERRENO,0,TERRENO,null
30008900636,AV ULYSSES REIS DE MATTOS,100,APTO 504 TORRE I,nan,nan,5686901,1.Compra e venda,2299955.7,2021-12-23T00:00:00.000Z,4.2424524E7,0.04,16969.8096,2299955.7,nan,0.0,15º Cartório de Registro de Imóvel,256974,Ativo Predial,9967,212.25,1.0,2049,31,"PRÉDIO DE ESCRITÓRIO OU CONSULTÓRIO, NÃO EM CONDOMÍNIO, COM OU SEM LOJAS",33,COMERCIAL HORIZONTAL,2018.0
2806802059,AV PRES WILSON,274,T3 AP2006,MOOCA,nan,3107900,1.Compra e venda,250000.0,2021-12-23T00:00:00.000Z,4.06875586E8,0.01,40687.5586,250000.0,1.Sistema Financeiro de Habitação,182723.79,7º Cartório de Registro de Imóvel,212252,Ativo Predial,146411,0.0,1.0,90609,50,INDÚSTRIA,84,"OFICINA, POSTO DE SERVIÇO, ARMAZÉM/DEPÓSITO, INDÚSTRIA",1973.0
2806802059,AV PRES WILSON,274,T2 AP1902,MOOCA,nan,3107900,1.Compra e venda,264000.0,2021-12-23T00:00:00.000Z,4.06875586E8,0.01,40687.5586,264000.0,1.Sistema Financeiro de Habitação,188333.65,7º Cartório de Registro de Imóvel,212252,Ativo Predial,146411,0.0,1.0,90609,50,INDÚSTRIA,84,"OFICINA, POSTO DE SERVIÇO, ARMAZÉM/DEPÓSITO, INDÚSTRIA",1973.0
2806802059,AV PRES WILSON,274,T2 AP1508,MOOCA,nan,3107900,1.Compra e venda,264000.0,2021-12-23T00:00:00.000Z,4.06875586E8,0.01,40687.5586,264000.0,1.Sistema Financeiro de Habitação,209000.0,7º Cartório de Registro de Imóvel,212252,Ativo Predial,146411,0.0,1.0,90609,50,INDÚSTRIA,84,"OFICINA, POSTO DE SERVIÇO, ARMAZÉM/DEPÓSITO, INDÚSTRIA",1973.0
2806802059,AV PRES WILSON,274,T2 AP1704,MOOCA,nan,3107900,1.Compra e venda,264000.0,2021-12-23T00:00:00.000Z,4.06875586E8,0.01,40687.5586,264000.0,1.Sistema Financeiro de Habitação,161000.0,7º Cartório de Registro de Imóvel,212252,Ativo Predial,146411,0.0,1.0,90609,50,INDÚSTRIA,84,"OFICINA, POSTO DE SERVIÇO, ARMAZÉM/DEPÓSITO, INDÚSTRIA",1973.0
2806802113,AV PRES WILSON,30,TORRE 03 AP 1703,nan,nan,3107000,1.Compra e venda,264000.0,2021-12-23T00:00:00.000Z,1.3087429E7,0.02,2617.4858,264000.0,1.Sistema Financeiro de Habitação,208166.75,7º Cartório de Registro de Imóvel,212252,Ativo Territorial,5591,140.73,1.0,0,0,TERRENO,0,TERRENO,null


In [0]:
%sql
/* Criação da tabela agregada contendo todos os dados de itbi desde 2022 */

CREATE OR REPLACE TABLE mvp_engdados_puc.bronze.guias_itbi_bronze AS

SELECT 
  try_cast(No_do_Cadastro__SQL_ AS BIGINT) AS No_do_Cadastro__SQL_,
  Nome_do_Logradouro,
  try_cast(`Número` AS BIGINT) AS `Número`,
  Complemento,
  Bairro,
  `Referência`,
  try_cast(CEP AS DOUBLE) AS CEP,
  `Natureza_de_Transação`,
  try_cast(`Valor_de_Transação__declarado_pelo_contribuinte_` AS DOUBLE) AS `Valor_de_Transação__declarado_pelo_contribuinte_`,
  try_cast(`Data_de_Transação` AS DATE) AS `Data_de_Transação`,
  try_cast(`Valor_Venal_de_Referência` AS DOUBLE) AS `Valor_Venal_de_Referência`,
  try_cast(`Proporção_Transmitida__pct_` AS DOUBLE) AS `Proporção_Transmitida__pct_`,
  try_cast(`Valor_Venal_de_Referência__proporcional_` AS DOUBLE) AS `Valor_Venal_de_Referência__proporcional_`,
  try_cast(`Base_de_Cálculo_adotada` AS DOUBLE) AS `Base_de_Cálculo_adotada`,
  Tipo_de_financiamento,
  try_cast(Valor_Financiado AS DOUBLE) AS Valor_Financiado,
  `Cartório_de_Registro`,
  try_cast(`Matrícula_do_Imóvel` AS BIGINT) AS `Matrícula_do_Imóvel`,
  `Situação_do_SQL`,
  try_cast(`Área_do_Terreno__m2_` AS BIGINT) AS `Área_do_Terreno__m2_`,
  try_cast(Testada__m_ AS BIGINT) AS Testada__m_,
  try_cast(`Fração_Ideal` AS DOUBLE) AS `Fração_Ideal`,
  try_cast(`Área_Construída__m2_` AS BIGINT) AS `Área_Construída__m2_`,
  try_cast(Uso__IPTU_ AS BIGINT) AS Uso__IPTU_,
  `Descrição_do_uso__IPTU_`,
  try_cast(`Padrão__IPTU_` AS BIGINT) AS `Padrão__IPTU_`,
  `Descrição_do_Padrão__IPTU_`,
  try_cast(ACC__IPTU_ AS STRING) AS ACC__IPTU_
  
FROM mvp_engdados_puc.bronze.guias_itbi_2025

UNION ALL
SELECT 
try_cast(No_do_Cadastro__SQL_ AS BIGINT) AS No_do_Cadastro__SQL_,
  Nome_do_Logradouro,
  try_cast(`Número` AS BIGINT) AS `Número`,
  Complemento,
  Bairro,
  `Referência`,
  try_cast(CEP AS DOUBLE) AS CEP,
  `Natureza_de_Transação`,
  try_cast(`Valor_de_Transação__declarado_pelo_contribuinte_` AS DOUBLE) AS `Valor_de_Transação__declarado_pelo_contribuinte_`,
  try_cast(`Data_de_Transação` AS DATE) AS `Data_de_Transação`,
  try_cast(`Valor_Venal_de_Referência` AS DOUBLE) AS `Valor_Venal_de_Referência`,
  try_cast(`Proporção_Transmitida__pct_` AS DOUBLE) AS `Proporção_Transmitida__pct_`,
  try_cast(`Valor_Venal_de_Referência__proporcional_` AS DOUBLE) AS `Valor_Venal_de_Referência__proporcional_`,
  try_cast(`Base_de_Cálculo_adotada` AS DOUBLE) AS `Base_de_Cálculo_adotada`,
  Tipo_de_financiamento,
  try_cast(Valor_Financiado AS DOUBLE) AS Valor_Financiado,
  `Cartório_de_Registro`,
  try_cast(`Matrícula_do_Imóvel` AS BIGINT) AS `Matrícula_do_Imóvel`,
  `Situação_do_SQL`,
  try_cast(`Área_do_Terreno__m2_` AS BIGINT) AS `Área_do_Terreno__m2_`,
  try_cast(Testada__m_ AS BIGINT) AS Testada__m_,
  try_cast(`Fração_Ideal` AS DOUBLE) AS `Fração_Ideal`,
  try_cast(`Área_Construída__m2_` AS BIGINT) AS `Área_Construída__m2_`,
  try_cast(Uso__IPTU_ AS BIGINT) AS Uso__IPTU_,
  `Descrição_do_uso__IPTU_`,
  try_cast(`Padrão__IPTU_` AS BIGINT) AS `Padrão__IPTU_`,
  ACC__IPTU_ AS `Descrição_do_Padrão__IPTU_`, 
  try_cast(ACC__IPTU__1 AS STRING) AS ACC__IPTU_

FROM mvp_engdados_puc.bronze.guias_itbi_1

UNION ALL
SELECT 
try_cast(No_do_Cadastro__SQL_ AS BIGINT) AS No_do_Cadastro__SQL_,
  Nome_do_Logradouro,
  try_cast(`Número` AS BIGINT) AS `Número`,
  Complemento,
  Bairro,
  `Referência`,
  try_cast(CEP AS DOUBLE) AS CEP,
  `Natureza_de_Transação`,
  try_cast(`Valor_de_Transação__declarado_pelo_contribuinte_` AS DOUBLE) AS `Valor_de_Transação__declarado_pelo_contribuinte_`,
  try_cast(`Data_de_Transação` AS DATE) AS `Data_de_Transação`,
  try_cast(`Valor_Venal_de_Referência` AS DOUBLE) AS `Valor_Venal_de_Referência`,
  try_cast(`Proporção_Transmitida__pct_` AS DOUBLE) AS `Proporção_Transmitida__pct_`,
  try_cast(`Valor_Venal_de_Referência__proporcional_` AS DOUBLE) AS `Valor_Venal_de_Referência__proporcional_`,
  try_cast(`Base_de_Cálculo_adotada` AS DOUBLE) AS `Base_de_Cálculo_adotada`,
  Tipo_de_financiamento,
  try_cast(Valor_Financiado AS DOUBLE) AS Valor_Financiado,
  `Cartório_de_Registro`,
  try_cast(`Matrícula_do_Imóvel` AS BIGINT) AS `Matrícula_do_Imóvel`,
  `Situação_do_SQL`,
  try_cast(`Área_do_Terreno__m2_` AS BIGINT) AS `Área_do_Terreno__m2_`,
  try_cast(Testada__m_ AS BIGINT) AS Testada__m_,
  try_cast(`Fração_Ideal` AS DOUBLE) AS `Fração_Ideal`,
  try_cast(`Área_Construída__m2_` AS BIGINT) AS `Área_Construída__m2_`,
  try_cast(Uso__IPTU_ AS BIGINT) AS Uso__IPTU_,
  `Descrição_do_uso__IPTU_`,
  try_cast(`Padrão__IPTU_` AS BIGINT) AS `Padrão__IPTU_`,
  `Descrição_do_Padrão__IPTU_`,
  try_cast(ACC__IPTU_ AS STRING) AS ACC__IPTU_ 

FROM mvp_engdados_puc.bronze.guias_itbi_2

UNION ALL
SELECT 
try_cast(No_do_Cadastro__SQL_ AS BIGINT) AS No_do_Cadastro__SQL_,
  Nome_do_Logradouro,
  try_cast(`Número` AS BIGINT) AS `Número`,
  Complemento,
  Bairro,
  `Referência`,
  try_cast(CEP AS DOUBLE) AS CEP,
  `Natureza_de_Transação`,
  try_cast(`Valor_de_Transação__declarado_pelo_contribuinte_` AS DOUBLE) AS `Valor_de_Transação__declarado_pelo_contribuinte_`,
  try_cast(`Data_de_Transação` AS DATE) AS `Data_de_Transação`,
  try_cast(`Valor_Venal_de_Referência` AS DOUBLE) AS `Valor_Venal_de_Referência`,
  try_cast(`Proporção_Transmitida__pct_` AS DOUBLE) AS `Proporção_Transmitida__pct_`,
  try_cast(`Valor_Venal_de_Referência__proporcional_` AS DOUBLE) AS `Valor_Venal_de_Referência__proporcional_`,
  try_cast(`Base_de_Cálculo_adotada` AS DOUBLE) AS `Base_de_Cálculo_adotada`,
  Tipo_de_financiamento,
  try_cast(Valor_Financiado AS DOUBLE) AS Valor_Financiado,
  `Cartório_de_Registro`,
  try_cast(`Matrícula_do_Imóvel` AS BIGINT) AS `Matrícula_do_Imóvel`,
  `Situação_do_SQL`,
  try_cast(`Área_do_Terreno__m2_` AS BIGINT) AS `Área_do_Terreno__m2_`,
  try_cast(Testada__m_ AS BIGINT) AS Testada__m_,
  try_cast(`Fração_Ideal` AS DOUBLE) AS `Fração_Ideal`,
  try_cast(`Área_Construída__m2_` AS BIGINT) AS `Área_Construída__m2_`,
  try_cast(Uso__IPTU_ AS BIGINT) AS Uso__IPTU_,
  `Descrição_do_uso__IPTU_`,
  try_cast(`Padrão__IPTU_` AS BIGINT) AS `Padrão__IPTU_`,
  ACC__IPTU_ `Descrição_do_Padrão__IPTU_`,
  try_cast(ACC__IPTU__1 AS STRING) AS ACC__IPTU_
  
FROM mvp_engdados_puc.bronze.guias_itbi_3

num_affected_rows,num_inserted_rows


In [0]:
%sql
/* Visualizacão da tabela criada */

SELECT * FROM mvp_engdados_puc.bronze.guias_itbi_bronze
LIMIT 20

No_do_Cadastro__SQL_,Nome_do_Logradouro,Número,Complemento,Bairro,Referência,CEP,Natureza_de_Transação,Valor_de_Transação__declarado_pelo_contribuinte_,Data_de_Transação,Valor_Venal_de_Referência,Proporção_Transmitida__pct_,Valor_Venal_de_Referência__proporcional_,Base_de_Cálculo_adotada,Tipo_de_financiamento,Valor_Financiado,Cartório_de_Registro,Matrícula_do_Imóvel,Situação_do_SQL,Área_do_Terreno__m2_,Testada__m_,Fração_Ideal,Área_Construída__m2_,Uso__IPTU_,Descrição_do_uso__IPTU_,Padrão__IPTU_,Descrição_do_Padrão__IPTU_,ACC__IPTU_
13944000322,AV NORDESTINA,3010,AP 918 TORRE 2,VL CURUCA,nan,8032000.0,1.Compra e venda,264000.0,2022-12-22,1.582887E7,0.2,31657.74,264000.0,1.Sistema Financeiro de Habitação,166320.0,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
6726500084,R JURIPIRANGA,70,ANT 16,PARQUE VITORIA,nan,2269090.0,1.Compra e venda,800000.0,2022-12-23,846885.0,100.0,846885.0,846885.0,1.Sistema Financeiro de Habitação,400000.0,15º Cartório de Registro de Imóvel,87025,Ativo Predial,210,10,1.0,277,10,RESIDÊNCIA,12,RESIDENCIAL HORIZONTAL,2009.0
13944000322,AV NORDESTINA,3010,AP 504 SUB 2 BL 2,VL CURUCA,nan,8032000.0,1.Compra e venda,203700.0,2022-12-22,1.582887E7,0.01,1582.887,203700.0,1.Sistema Financeiro de Habitação,144059.51,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
5614900926,R ANTONIO DE BARROS,99999,AP 614 BL 3,TATUAPE,nan,3401001.0,1.Compra e venda,264000.0,2022-12-22,3403192.0,0.08,2722.5536,264000.0,1.Sistema Financeiro de Habitação,183400.0,9º Cartório de Registro de Imóvel,336376,Ativo Territorial,1000,20,1.0,0,0,TERRENO,0,TERRENO,nan
13944000322,AV NORDESTINA,3010,AP 305 SUB 1 BL 1,VL CURUCA,nan,8032000.0,1.Compra e venda,200550.0,2022-12-22,1.582887E7,0.01,1582.887,200550.0,1.Sistema Financeiro de Habitação,100433.0,12º Cartório de Registro de Imóvel,236745,Ativo Predial,11131,0,1.0,1400,40,LOJA,32,COMERCIAL HORIZONTAL,2008.0
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 2102 T A,PIRITUBA,nan,2938000.0,1.Compra e venda,264000.0,2022-12-22,2.3872922E7,0.02,4774.5844,264000.0,1.Sistema Financeiro de Habitação,208635.41,16º Cartório de Registro de Imóvel,185609,Ativo Territorial,27500,172,1.0,0,0,TERRENO,0,TERRENO,nan
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 1701 T B,PIRITUBA,nan,2938000.0,1.Compra e venda,264000.0,2022-12-22,2.3872922E7,0.02,4774.5844,264000.0,1.Sistema Financeiro de Habitação,191064.56,16º Cartório de Registro de Imóvel,185610,Ativo Territorial,27500,172,1.0,0,0,TERRENO,0,TERRENO,nan
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 1712 T A,PIRITUBA,nan,2938000.0,1.Compra e venda,260000.0,2022-12-22,2.3872922E7,0.02,4774.5844,260000.0,1.Sistema Financeiro de Habitação,165514.76,16º Cartório de Registro de Imóvel,185608,Ativo Territorial,27500,172,1.0,0,0,TERRENO,0,TERRENO,nan
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 505 T A,PIRITUBA,nan,2938000.0,1.Compra e venda,264000.0,2022-12-22,2.3872922E7,0.02,4774.5844,264000.0,1.Sistema Financeiro de Habitação,211200.0,16º Cartório de Registro de Imóvel,185610,Ativo Territorial,27500,172,1.0,0,0,TERRENO,0,TERRENO,nan
7700100012,AV RAIMUNDO PEREIRA DE MAGALHAES,5028,AP 2104 T A,PIRITUBA,nan,2938000.0,1.Compra e venda,264000.0,2022-12-22,2.3872922E7,0.02,4774.5844,264000.0,1.Sistema Financeiro de Habitação,211200.0,16º Cartório de Registro de Imóvel,185608,Ativo Territorial,27500,172,1.0,0,0,TERRENO,0,TERRENO,nan


In [0]:
%sql
/* Contador do número de linhas da tabela de itbi, camada bronze */
SELECT COUNT(*) FROM mvp_engdados_puc.bronze.guias_itbi_bronze

COUNT(*)
745179


### 3.2.2 Tabela de dados de Uso do imóvel segundo o IPTU:

In [0]:
# Cria tabela com dados da dimensão Uso do IPTU

import requests
import pandas as pd
import re

url = (
    'https://prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/itbi/'
    'GUIAS%20DE%20ITBI%20PAGAS%20%2825112025%29.xlsx'
)

# Baixa o arquivo da url e salva em um arquivo temporário no cluster
response = requests.get(url)
with open('/tmp/seuarquivo.xlsx', 'wb') as f:
    f.write(response.content)

# Lê apenas a aba chamada "Tabela de USOS"
df_usos = pd.read_excel(
    '/tmp/seuarquivo.xlsx',
    sheet_name="Tabela de USOS"
)

# Seleciona apenas as 2 primeiras colunas, pois se existirem mais colunas são lixo
df_usos = df_usos.iloc[:, :2].copy()

# Converte todas as colunas que contêm texto para tipo de dado string
for col in df_usos.select_dtypes(include=['object']).columns:
    df_usos[col] = df_usos[col].astype(str)

# Converte o DataFrame do Pandas para um DataFrame do Spark
df_spark_usos = spark.createDataFrame(df_usos)

# Deleta a tabela, caso já exista
table_name_usos = "mvp_engdados_puc.bronze.tabela_dim_usos"
spark.sql(f"DROP TABLE IF EXISTS {table_name_usos} PURGE")

# Conta número de linhas do DataFrame e mostra no máximo 100 linhas dele
num_linhas_usos = df_spark_usos.count()
print(f"Número de linhas no Spark: {num_linhas_usos}")
display(df_spark_usos.limit(100))

# Salva o DataFrame do Spark em uma tabela persistida no DBFS
df_spark_usos.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(table_name_usos)

Número de linhas no Spark: 36


USO,DESCRIÇÃO
0,TERRENO
10,RESIDÊNCIA
12,"RESIDÊNCIA COLETIVA, EXCLUSIVE CORTIÇO (MAIS DE UMA RESIDÊNCIA NO LOTE)"
13,CORTIÇO (HABITAÇÃO COLETIVA SUBNORMAL)
14,RESIDÊNCIA E OUTRO USO (PREDOMINÂNCIA RESIDENCIAL)
20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL)
21,"PRÉDIO DE APARTAMENTO, NÃO EM CONDOMÍNIO, DE USO EXCLUSIVAMENTE RESIDENCIAL"
22,"PRÉDIO DE APARTAMENTO, NÃO EM CONDOMÍNIO, DE USO MISTO (APARTAMENTOS E ESCRITÓRIOS E/OU CONSULTÓRIOS), COM OU SEM LOJA (PREDOMINÂNCIA RESIDENCIAL)"
23,"GARAGEM (UNIDADE AUTÔNOMA) EM EDIFÍCIO EM CONDOMÍNIO DE ESCRITÓRIOS, CONSULTÓRIOS OU MISTO (EXIGE FRAÇÃO IDEAL)"
24,GARAGEM (UNIDADE AUTÔNOMA) EM EDIFÍCIO EM CONDOMÍNIO DE USO EXCLUSIVAMENTE RESIDENCIAL (EXIGE FRAÇÃO IDEAL)


In [0]:
%sql
/* Visualizacão da tabela criada */

SELECT * FROM mvp_engdados_puc.bronze.tabela_dim_usos

USO,DESCRIÇÃO
0,TERRENO
10,RESIDÊNCIA
12,"RESIDÊNCIA COLETIVA, EXCLUSIVE CORTIÇO (MAIS DE UMA RESIDÊNCIA NO LOTE)"
13,CORTIÇO (HABITAÇÃO COLETIVA SUBNORMAL)
14,RESIDÊNCIA E OUTRO USO (PREDOMINÂNCIA RESIDENCIAL)
20,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL)
21,"PRÉDIO DE APARTAMENTO, NÃO EM CONDOMÍNIO, DE USO EXCLUSIVAMENTE RESIDENCIAL"
22,"PRÉDIO DE APARTAMENTO, NÃO EM CONDOMÍNIO, DE USO MISTO (APARTAMENTOS E ESCRITÓRIOS E/OU CONSULTÓRIOS), COM OU SEM LOJA (PREDOMINÂNCIA RESIDENCIAL)"
23,"GARAGEM (UNIDADE AUTÔNOMA) EM EDIFÍCIO EM CONDOMÍNIO DE ESCRITÓRIOS, CONSULTÓRIOS OU MISTO (EXIGE FRAÇÃO IDEAL)"
24,GARAGEM (UNIDADE AUTÔNOMA) EM EDIFÍCIO EM CONDOMÍNIO DE USO EXCLUSIVAMENTE RESIDENCIAL (EXIGE FRAÇÃO IDEAL)


### 3.2.3 Tabela com dados dos CEPs de São Paulo:

A ingestão dos dados de CEP foi feita manualmente pelo Unit Catalog. Evidências abaixo:

![](Figuras/Ingestao_UnitCatalog_1)

![](Figuras/Ingestao_UnitCatalog_2)

![](Figuras/Ingestao_UnitCatalog_3)

![](Figuras/Ingestao_UnitCatalog_4)

In [0]:
# Criação da tabela com todos os CEPs do estado de São Paulo
import zipfile
from functools import reduce

zip_paths = [
    "/Volumes/mvp_engdados_puc/staging/bases_cep_sp/sp.cepaberto_parte_1.zip",
    "/Volumes/mvp_engdados_puc/staging/bases_cep_sp/sp.cepaberto_parte_2.zip",
    "/Volumes/mvp_engdados_puc/staging/bases_cep_sp/sp.cepaberto_parte_3.zip",
    "/Volumes/mvp_engdados_puc/staging/bases_cep_sp/sp.cepaberto_parte_4.zip",
    "/Volumes/mvp_engdados_puc/staging/bases_cep_sp/sp.cepaberto_parte_5.zip"
]
staging_dir = "/Volumes/mvp_engdados_puc/staging/bases_cep_sp"

csv_paths = []
# Para cada um dos arquivos zip salvos em staging_dir, extrai o único arquivo CSV que ele contém e salva em csv_paths
for zip_path in zip_paths:    
    with zipfile.ZipFile(zip_path, 'r') as arquivo_zip:
        nome_do_csv = arquivo_zip.namelist()[0]
        arquivo_zip.extract(nome_do_csv, staging_dir)
        csv_paths.append(f"{staging_dir}/{nome_do_csv}")

# Header para os CSVs
colunas_cep = ["cep", "logradouro", "descricao", "bairro", "cidade", "uf"]

# Lê cada um dos CSV, que não tem header, forçando o esquema indicado em colunas_cep
df_list = [
    spark.read.csv(
        path,
        sep=",",
        header=False,
        inferSchema=False
    ).toDF(*colunas_cep)
    for path in csv_paths
]

# Lê a lista de dataframes armazenados em df_list e une todos em um só
df_cep_unido = reduce(lambda df1, df2: df1.unionByName(df2), df_list)

# Deleta a tabela, caso já exista
table_name_cep = "mvp_engdados_puc.bronze.cepaberto_SP"
spark.sql(f"DROP TABLE IF EXISTS {table_name_cep} PURGE")

# Salva o DataFrame do Spark em uma tabela Delta persistida no DBFS
df_cep_unido.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(table_name_cep)

# Comando para ler 20 linhas da tabela Delta criada
display(spark.table(table_name_cep).limit(20))

cep,logradouro,descricao,bairro,cidade,uf
19915430,Rua A,null,Jardim Santa Catarina,6404,26
19915433,Rua Dois,null,Jardim Santa Catarina,6404,26
19915450,Rua Valdir Martins Tavares,null,Chácaras Bom Retiro,6404,26
19915480,Rua Milton de Abreu,null,Conjunto Habitacional Orlando Quagliato,6404,26
19915481,Rua Dois,null,Conjunto Habitacional Orlando Quagliato,6404,26
19915484,Rua Ezelino Zório Maulen,null,Conjunto Habitacional Orlando Quagliato,6404,26
19915485,Rua Silmara Bolognese da Silva,null,Conjunto Habitacional Orlando Quagliato,6404,26
19915486,Rua Pedro Fantinatti,null,Conjunto Habitacional Orlando Quagliato,6404,26
19915487,Rua Emílio Roli,null,Conjunto Habitacional Orlando Quagliato,6404,26
19915488,Rua Corino Martins de Oliveira,null,Conjunto Habitacional Orlando Quagliato,6404,26


In [0]:
# Criação da tabela de códigos das cidades (base CEP)
import zipfile

zip_path = "/Volumes/mvp_engdados_puc/staging/bases_cep_sp/cidades.cepaberto.zip"
staging_dir = "/Volumes/mvp_engdados_puc/staging/bases_cep_sp"

# Extrai o único arquivo CSV contido no zip_path e salva no diretório
with zipfile.ZipFile(zip_path, 'r') as arquivo_zip:
    nome_do_csv = arquivo_zip.namelist()[0]
    arquivo_zip.extract(nome_do_csv, staging_dir)
    csv_path = f"{staging_dir}/{nome_do_csv}"

# Header para o CSV
colunas_cidades = ["codigo_cidade", "nome_cidade", "codigo_estado"]

# Lê o CSV, que não tem header, forçando o esquema indicado em colunas_cidades
df_cidades = spark.read.csv(
    csv_path,
    sep=",",
    header=False,
    inferSchema=False
).toDF(*colunas_cidades)

# Deleta a tabela, caso já exista
table_name = "mvp_engdados_puc.bronze.cepaberto_cidades"
spark.sql(f"DROP TABLE IF EXISTS {table_name} PURGE")

# Salva o DataFrame do Spark em uma tabela Delta persistida no DBFS
df_cidades.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(table_name)

# Comando para ler 100 linhas da tabela Delta criada
display(spark.table(table_name).limit(20))

codigo_cidade,nome_cidade,codigo_estado
1,Abacate da Pedreira (Macapá),4
2,Abadia de Goiás,9
3,Abadia dos Dourados,11
4,Abadia (Jandaíra),5
5,Abadiânia,9
6,Abaeté,11
7,Abaeté dos Mendes (Rio Paranaíba),11
8,Abaetetuba,14
9,Abaiara,6
10,Abaíba (Leopoldina),11
